In [3]:
import pandas as pd
import numpy as np
import json
from tools import calculate_similarity, calculate_performance, calculate_performance_V2, pdf_to_text
import os
from difflib import SequenceMatcher

In [ ]:
import requests
import json

def get_crossref_metadata(title):
    url = "https://api.crossref.org/works"
    params = {
        "query.bibliographic": title,
        "rows": 1  # return the top result
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        items = data.get("message", {}).get("items", [])
        if items:
            return items[0]
    return None

def reduce_metadata(metadata):
    # Extract the title from the 'title' list (if available)
    title_list = metadata.get("title", [])
    title = title_list[0] if title_list else "NA"
    
    # Extract abstract (if available)
    abstract = metadata.get("abstract", "NA")
    
    # Extract date published from the 'issued' field
    issued = metadata.get("issued", {})
    date_parts = issued.get("date-parts", [])
    if date_parts and len(date_parts[0]) > 0:
        date_published = "-".join(str(part) for part in date_parts[0])
    else:
        date_published = "NA"
        
    # Extract DOI
    doi = metadata.get("DOI", "NA")
    ref_count = metadata.get("reference-count", "NA")
    citations = metadata.get("is-referenced-by-count", "NA")
    container_title = metadata.get("container-title", [])
    journal = container_title[0] if container_title else "NA"
    date_updated = metadata.get("indexed", {}).get("date-time", "NA")
    
    return {
        "Title": title,
        "Abstract": abstract,
        "Date Published": date_published,
        "DOI": doi,
        "Number of References": ref_count,
        "Number of Citations": citations,
        "Journal": journal,
        "Date Updated (of metadata)": date_updated
    }

def text_similarity(text1, text2):
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()




In [5]:
json_file = 'outputs/articlesMetaData.json'
output_json_path = "outputs/articlesMetaDataCrossRef.json"
if os.path.exists(json_file):
    print('Skipping Creation: File Already Exists')
else:
    df = pd.read_excel('documents/articles.xlsx', sheet_name='6061 Al')

    data = []
    for index, row in df.iterrows():
        article = {
            "filename": f"Article_{row['Index']}.pdf",
            "title": "" if pd.isna(row['Title']) else row['Title'],
            "link": "" if pd.isna(row['Link']) else row['Link'],
            "extractedText": ""
        }
        data.append(article)
    with open(json_file, 'w') as f:
        json.dump(data, f, indent=4)

    
    '''------------------------------------------------------------------'''
    # Read the JSON file
    with open(json_file, 'r', encoding='utf-8') as infile:
        articles = json.load(infile)

    # Process each article: get metadata from Crossref and add a new "metadata" field
    for article in articles:
        article_title = article.get("title", "")
        if article_title:
            crossref_metadata = get_crossref_metadata(article_title)
            if crossref_metadata:
                reduced = reduce_metadata(crossref_metadata)
                returned_title = reduced.get("Title", "NA")
                similarity = text_similarity(article_title, returned_title)
                if similarity < 0.9:
                    article["metadata"] = "No metadata found"
                else:
                    article["metadata"] = reduced
            else:
                article["metadata"] = "No metadata found"
        else:
            article["metadata"] = "No title provided"

    # Save the new JSON to a file (you can change the output path as needed)
    
    with open(output_json_path, 'w', encoding='utf-8') as outfile:
        json.dump(articles, outfile, indent=4)

    print(f"Processed articles saved to {output_json_path}")


Skipping Creation: File Already Exists


In [6]:
from openai import OpenAI
with open("../openAiToken.txt", "r") as key_file:
    api_key = key_file.read().strip()
os.environ["OPENAI_API_KEY"] = api_key  # Optionally set it as an environment variable
client = OpenAI(api_key=api_key)

In [15]:
MODEL = "o3-mini"
EFFORT = 'high'
MATERIAL = 'Al6061'
INPUT_FILE = f"prompts/reportedValueExtraction.txt"
OUTPUT_FILE = f'outputs/{MODEL}/reportedValueExtraction_{EFFORT}_{MATERIAL}_3-25v3.json'
JSON_FILE = "outputs/articlesMetaDataCrossRef.json"

with open(JSON_FILE) as f:
    articles = json.load(f)

for article in articles[:5]:
    articlePresent = False
    print(article['filename'])
    if os.path.exists(f"documents/{MATERIAL}/{article['filename']}"): 
        articlePresent = True
    else:
        print('Skipping Because Article not found')
        print(f"documents/{MATERIAL}/{article['filename']}")

    if articlePresent:
        print(f'{article["filename"]} Ready for Analysis')
        input_pdf = f"documents/{MATERIAL}/{article['filename']}"
        article_text = pdf_to_text(input_pdf)
        with open(INPUT_FILE, "r") as file:
            prompt_text = file.read()
        messages=[ 
            {"role": "user", "content": prompt_text},
            {
                "role": "user",
                "content": article_text
            }
        ]
        try:
            completion = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                reasoning_effort=EFFORT
            )
        except Exception as e:
            # Check for an error message that indicates the context is too large.
            if "string too long" in str(e).lower() or "token" in str(e).lower():
                print("Too large of Input")
                continue
            else:
                raise e

        extractedVals = completion.choices[0].message.content
        print(extractedVals)
        
        if extractedVals.startswith("```json") and extractedVals.endswith("```"):
            extractedVals = extractedVals[7:-3].strip()

        try:
            article['extractedText'] = json.loads(extractedVals)
        except json.JSONDecodeError as e:
            print(f"Saving as TXT - Error decoding JSON: {e}")
            article['extractedText'] = extractedVals
        except Exception as e:
            print("Another Error Occured")
            raise e
            # article['extractedText'] = extractedVals
        # article['extractedText'] = json.loads(extractedVals)
        with open(OUTPUT_FILE, 'w') as f:
            json.dump(articles, f, indent=4)



Article_1.pdf
Article_1.pdf Ready for Analysis
{
  "pre_sprayed_material_properties": {
    "Powder Material Name": "A generic identifier for the powder feedstock material used in the spray process.",
    "Powder Material Composition": "A general description of the elemental or alloy composition of the powder feedstock.",
    "Powder Material Source": "Information regarding the supplier or manufacturer of the powder feedstock.",
    "Powder Specification": "The standard classification or specification of the powder, including criteria such as mesh size and military standard.",
    "Powder Particle Size Distribution": "The distribution of particle sizes in the powder feedstock, typically provided as D10, D50, and D90 values.",
    "Substrate Material Name": "A generic identifier for the substrate material onto which the deposit is applied.",
    "Substrate Material Composition": "A general description of the elemental or alloy makeup of the substrate.",
    "Substrate Thickness": "The p

In [20]:
import pdfplumber

def pdf_to_text(pdf_path):
    """
    Extract text from a PDF and return it as a single string.
    """
    full_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                full_text.append(text)
    return "\n".join(full_text)

# Example usage
if __name__ == "__main__":
    input_pdf = "documents/Al6061/Article_10.pdf"
    article_text = pdf_to_text(input_pdf)

    # At this point, `article_text` contains the extracted text from the PDF.
    # You can now perform any NLP preprocessing you like—such as cleaning,
    # splitting into chunks, summarization, etc.—before sending to an LLM.
    # For example:
    # processed_text = some_preprocessing_function(article_text)
    # send_to_llm(processed_text)

    print(article_text)


Applied Surface Science 534 (2020) 147643
Contents lists available at ScienceDirect
Applied Surface Science
journal homepage: www.elsevier.com/locate/apsusc
Surface states of gas-atomized Al 6061 powders – Effects of heat treatment
T
Alexis T. Ernsta, Peter Kernsb, Aaron Nardic, Harold D. Brodya, Avinash M. Dongarea,
Seok-Woo Leea, Victor K. Champagnec, Steven L. Suiba,b, Mark Aindowa,⁎
a Department of Materials Science and Engineering, Institute of Materials Science, University of Connecticut, Unit 3136, 97 North Eagleville Road, Storrs, CT 06269-3136,
USA
b Department of Chemistry, University of Connecticut, 55 North Eagleville Road, Unit 3060 Storrs, CT 06269-3060, USA
c U.S. Army Research Laboratory, Weapons and Materials Research Directorate, Aberdeen Proving Ground, Aberdeen, MD 21005, USA
ARTICLE INFO ABSTRACT
Keywords: Surface oxides formed on powder feedstocks used for cold spray deposition can play an important role in the
Aluminum alloy powder bonding of the particles and in

In [ ]:
Here is a large json. 

Focus on the extractedText field (with four sub categories: pre_sprayed_material_properties, experimental_parameters, sprayed_properties, and standards). 

I would like you to create a single json with these four dictinoaries aggregated together across all the different articles. In cases of duplicates, or really similar, only add one to the aggregate dataset. If there are any that would better fit in a different group (for example feedstock material in experimental conditions should more realistically be in pre_sprayed_material_properties), you may move these

[
    {
        "filename": "Article_1.pdf",
        "title": "Bulk fracture anisotropy in Cold-Sprayed Al 6061 deposits ...",
        "link": "https://www.sciencedirect.com/science/article/abs/pii/S0013794422000649",
        "extractedText": {
            "pre_sprayed_material_properties": {
                "Powder Material Name": "A generic identifier for the powder feedstock material used in the spray process.",
                "Powder Material Composition": "A general description of the elemental or alloy composition of the powder feedstock.",
                "Powder Material Source": "Information regarding the supplier or manufacturer of the powder feedstock.",
                "Powder Specification": "The standard classification or specification of the powder, including criteria such as mesh size and military standard.",
                "Powder Particle Size Distribution": "The distribution of particle sizes in the powder feedstock, typically provided as D10, D50, and D90 values.",
                "Substrate Material Name": "A generic identifier for the substrate material onto which the deposit is applied.",
                "Substrate Material Composition": "A general description of the elemental or alloy makeup of the substrate.",
                "Substrate Thickness": "The physical thickness of the substrate material used in the spraying process."
            },
            "experimental_parameters": {
                "Spraying System Model": "The model and make of the spraying system employed to perform the cold spray deposition.",
                "Powder Feed Rate": "The mass flow rate at which the powder is delivered to the spray system.",
                "Gas Type": "The type of gas used as the accelerating medium in the cold spray process.",
                "Gas Pressure": "The operating pressure of the accelerating gas during the spray process.",
                "Gas Temperature": "The temperature at which the gas is maintained during spraying to affect particle acceleration.",
                "Powder Pre-treatment Temperature": "The temperature used to heat treat the powder prior to spraying in order to remove adsorbed moisture.",
                "Powder Pre-treatment Duration": "The duration for which the powder is subjected to the pre-treatment temperature before spraying.",
                "Nozzle Material": "The material from which the spray nozzle is constructed, impacting its durability and performance.",
                "Nozzle Model": "The specific model identifier of the spray nozzle used in the deposition process.",
                "Nozzle Throat Length": "The length of the narrowest section of the nozzle that influences particle acceleration.",
                "Nozzle Length": "The overall length of the nozzle, which affects the flow dynamics of the spray.",
                "Nozzle Throat Diameter": "The diameter of the nozzle\u2019s throat that governs the flow rate of gas and particles.",
                "Nozzle Exit Diameter": "The diameter at the exit of the nozzle that helps determine the spray pattern.",
                "Nozzle Standoff Distance": "The distance maintained between the nozzle exit and the substrate during spraying.",
                "Rastering Speed": "The speed at which the spray nozzle moves over the substrate during the deposition process.",
                "Rastering Step Size": "The incremental offset distance between successive passes of the spray nozzle.",
                "Particle Velocity": "The measured speed of the powder particles as they exit the nozzle and impact the substrate.",
                "Substrate Surface Preparation": "The method used to abrade and clean the substrate prior to spraying to promote proper adhesion.",
                "Fracture Specimen Characteristic Length": "The nominal dimension of the compact tension specimens used for fracture toughness testing.",
                "Crosshead Displacement Rate (Fracture Test)": "The rate at which the testing machine\u2019s crosshead moves during fracture toughness testing.",
                "Fatigue Pre-cracking Load Ratio": "The ratio of the minimum to maximum load applied during the fatigue pre-cracking phase.",
                "Fatigue Pre-cracking Frequency": "The frequency at which cyclic loading is applied during the fatigue pre-cracking process.",
                "Pre-crack Cycle Count": "The approximate number of loading cycles required to develop a sufficient pre-crack in the specimen.",
                "Tensile Test Crosshead Speed": "The constant speed at which the testing machine's crosshead moves during tensile testing.",
                "Specimen Gauge Dimensions": "The dimensions of the reduced (gage) section of the tensile test specimens where strain is measured.",
                "Extensometer Gauge Length": "The specified gauge length over which strain is recorded during tensile testing.",
                "Testing Machine Model": "The model of the universal testing apparatus used for both tensile and fracture testing."
            },
            "sprayed_properties": {
                "Porosity": "The percentage of void area within the sprayed deposit that indicates its density and consolidation.",
                "Elastic Modulus": "The measured stiffness of the deposit as determined from tensile testing.",
                "Yield Strength": "The stress at which the sprayed deposit begins to plastically deform under tension.",
                "Tensile Strength": "The maximum stress that the sprayed deposit can withstand before fracturing in tension.",
                "Elongation at Break": "The strain experienced by the deposit at the point of fracture, reflecting its ductility.",
                "Fracture Toughness (In-Plane)": "The resistance to crack propagation in the deposit when the crack is oriented within the deposition plane.",
                "Fracture Toughness (Out-of-Plane)": "The resistance to crack propagation when the crack is oriented perpendicular to the deposition plane.",
                "Fracture Mode Characterization": "A qualitative assessment describing whether fracture occurred predominantly along particle interfaces (interparticular) or through particles (transparticular)."
            },
            "standards": {
                "ASTM C633": "A standard test method for assessing the adhesion or cohesion strength of thermal spray coatings.",
                "ASTM E399": "A standard test method for determining the linear-elastic plane-strain fracture toughness of metallic materials.",
                "ASTM E561": "A standard test method for establishing the plane stress fracture resistance (KR curve) of metals.",
                "ASTM E1820": "A standard test method for measuring fracture toughness and crack resistance in elastic\u2013plastic materials.",
                "ASTM E647": "A standard test method for determining fatigue crack growth rates in materials.",
                "ASTM E8": "A standard test method for tension testing of metallic materials used to determine tensile properties.",
                "ASTM E111": "A standard test method for measuring Young's modulus and related elastic moduli in materials.",
                "ASTM E2109-01": "A standard test method for determining the area percentage porosity in thermal sprayed coatings.",
                "ASTM E3": "A standard guide for the preparation of metallographic specimens.",
                "ISO 12135": "An international standard describing a unified method for determining quasi-static fracture toughness."
            }
        },
        "metadata": {
            "Title": "Bulk fracture anisotropy in Cold-Sprayed Al 6061 deposits",
            "Abstract": "NA",
            "Date Published": "2022-3",
            "DOI": "10.1016/j.engfracmech.2022.108301",
            "Number of References": 73,
            "Number of Citations": 20,
            "Journal": "Engineering Fracture Mechanics",
            "Date Updated (of metadata)": "2024-09-20T16:52:38Z"
        }
    },
    {
        "filename": "Article_2.pdf",
        "title": "Aluminium 6061 - SPEE3D 3D Metal Printing",
        "link": "https://www.spee3d.com/material/aluminium-alloy-6061-spd/",
        "extractedText": "",
        "metadata": "No metadata found"
    },
    {
        "filename": "Article_3.pdf",
        "title": "Tailoring powder strengths for enhanced quality of cold sprayed Al6061 deposits",
        "link": "https://www.sciencedirect.com/science/article/pii/S0264127522001150",
        "extractedText": {
            "pre_sprayed_material_properties": {
                "Powder Material Name": "The generic identifier of the powder material used in the experiment.",
                "Powder Material Composition": "The elemental and phase constituents of the powder material.",
                "Powder Material Source": "The supplier or production method of the powder, typically via gas atomization.",
                "Powder Particle Size D10": "The particle size below which 10% of the powder volume is contained.",
                "Powder Particle Size D50": "The median particle size of the powder, representing the 50% volume distribution.",
                "Powder Particle Size D90": "The particle size below which 90% of the powder volume is contained.",
                "Powder Strength": "The inherent mechanical strength of the powder material as determined by compression tests.",
                "Powder Microstructure": "The structural characteristics of the powder particles, including grain size, phase distribution, and solute segregation.",
                "Powder Morphology": "The shape and surface features of the powder particles, including sphericity and the presence of satellites.",
                "Powder Oxygen Content": "The level of oxygen present on the powder surface, indicating the degree of oxidation.",
                "Powder Heat Treatment State": "The thermal history of the powder (e.g., as-atomized or annealed) that affects its microstructure and mechanical properties."
            },
            "experimental_parameters": {
                "Process Gas Type": "The type of gas used during the cold spraying process to accelerate the powder particles.",
                "Process Gas Temperature": "The temperature of the process gas, which preheats and accelerates the particles during spraying.",
                "Process Gas Pressure": "The pressure of the process gas, influencing particle acceleration and impact energy.",
                "Standoff Distance": "The distance between the spray nozzle and the substrate during deposition.",
                "Gun Traverse Speed": "The speed at which the spray gun moves across the substrate during the coating process.",
                "Powder Injection Feed Rate": "The rate at which the powder is delivered into the spray system.",
                "Powder Feeder Rotation Speed": "The rotational speed of the powder feeder that affects the consistency of powder delivery.",
                "Spray Equipment Model": "The model identifier of the cold spray system used for the deposition process.",
                "Spray Nozzle Type": "The configuration of the nozzle used to accelerate and direct the powder particles during spraying.",
                "Pre-Chamber Extension": "The extension added to the spray system to enhance particle pre-heating before impact."
            },
            "sprayed_properties": {
                "Deposition Efficiency": "The ratio of deposited material relative to the powder feedstock input, indicating the effectiveness of the cold spray process.",
                "Deposit Thickness": "The physical thickness of the coating achieved through cold spraying.",
                "Deposit Microhardness": "The measured resistance of the deposit to indentation, reflecting its micro-scale mechanical strength.",
                "Deposit Yield Strength": "The stress at which the deposit begins to deform plastically.",
                "Deposit Ultimate Tensile Strength": "The maximum stress that the deposited material can withstand before fracture.",
                "Deposit Ductility": "The ability of the deposit to undergo significant plastic deformation without fracturing.",
                "Deposit Porosity": "The fraction of void space within the deposit that affects its structural integrity and performance.",
                "Deposit Electrical Conductivity": "The capability of the deposit to conduct electrical current, indicative of material quality and bonding.",
                "Single-Particle Adhesion": "The adhesion strength of individual sprayed particles to the substrate as assessed by cavitation testing.",
                "Deposit Quality Parameter (g-value)": "A dimensionless parameter derived from powder strength and impact conditions that correlates with the overall deposit performance.",
                "Deposit Fracture Behavior": "The observed mode of failure and fracture characteristics in the deposited material."
            },
            "standards": {
                "ASTM E384-2": "A standard test method for measuring microhardness using a Vickers indenter.",
                "ASTM E1004": "A standard test method for determining electrical conductivity using an inductive technique.",
                "ASTM G32-10": "A standard test method for cavitation erosion testing to assess material performance under cavitation conditions."
            }
        },
        "metadata": {
            "Title": "Tailoring Powder Strengths for Enhanced Quality of Cold Sprayed Al6061 Deposits",
            "Abstract": "NA",
            "Date Published": "2022",
            "DOI": "10.2139/ssrn.4004791",
            "Number of References": 35,
            "Number of Citations": 0,
            "Journal": "SSRN Electronic Journal",
            "Date Updated (of metadata)": "2023-03-23T04:31:23Z"
        }
    },
    {
        "filename": "Article_4.pdf",
        "title": "Stress Corrosion Cracking Resistance of Cold-Sprayed Al 6061 Deposits Using a Newly Developed Test Fixture",
        "link": "https://www.mdpi.com/2079-6412/9/7/445",
        "extractedText": {
            "pre_sprayed_material_properties": {
                "Powder Material Name": "Represents the generic name of the feedstock powder used before spraying.",
                "Powder Material Composition": "Represents the alloy composition of the feedstock powder in generic terms.",
                "Powder Particle Size": "Represents the average size of the powder particles used as feedstock in the cold spray process.",
                "Feedstock Powder Grain Structure": "Represents the inherent grain structure of the powder prior to spraying that influences the final microstructure."
            },
            "experimental_parameters": {
                "Nozzle Type": "Specifies the type or design of the nozzle used in the cold spray process.",
                "Cold Spray Nozzle Throat Diameter": "Represents the diameter of the nozzle's throat, which affects particle acceleration.",
                "Exit Diameter": "Specifies the diameter at the exit of the nozzle that influences the particle flow and deposition characteristics.",
                "Nozzle Length": "Represents the length of the cold spray nozzle used during deposition.",
                "Nozzle Material": "Specifies the construction material of the nozzle, affecting its durability and performance.",
                "Gas Type": "Indicates the type of propellant gas used in the cold spray process.",
                "Gas Pressure": "Represents the operating pressure of the gas during spraying.",
                "Mass Feed Rate": "Specifies the rate at which the powder material is fed into the cold spray system by mass.",
                "Robot Program Raster": "Describes the programmed motion pattern of the robot during the deposition process.",
                "Raster Speed": "Represents the speed at which the robot moves during spraying.",
                "Standoff Distance": "Specifies the gap between the nozzle and the substrate during deposition.",
                "Deposition Efficiency": "Represents the efficiency with which the powder material is deposited onto the substrate.",
                "Coating Cost per Area": "Indicates the estimated cost of the coating process per unit area based on modeling.",
                "Coating Composition": "Represents the formulation or mixture of materials used to create the deposited coating.",
                "Tensile Test Strain Rate": "Specifies the rate at which strain is applied during tensile testing of the specimens.",
                "Extensometer Resolution": "Represents the measurement resolution of the extensometer used during mechanical testing.",
                "Load Cell Capacity": "Specifies the capacity of the load cell employed in the tensile testing setup.",
                "Tensile Specimen Dimensions": "Represents the geometric dimensions of the specimens used for tensile testing.",
                "Salt Fog Exposure Duration": "Specifies the time period samples are exposed to the salt fog environment during testing.",
                "Salt Fog Temperature": "Represents the temperature maintained during the salt fog exposure tests.",
                "Salt Fog pH": "Indicates the pH level of the salt solution used in the fog exposure testing.",
                "Salt Fog Concentration": "Specifies the concentration of salt in the fog environment during exposure.",
                "Test Fixture Design": "Represents the design and configuration parameters of the stress corrosion cracking test fixture.",
                "Fixture Calibration Method": "Describes the process used to calibrate the test fixture with sensors to ensure accurate load application."
            },
            "sprayed_properties": {
                "Coating Thickness": "Represents the uniform thickness of the cold-sprayed deposit after machining.",
                "Yield Strength": "Represents the yield strength of the sprayed deposit as determined by tensile testing.",
                "Tensile Strength": "Indicates the ultimate tensile strength of the sprayed deposit derived from mechanical testing.",
                "Elongation at Fracture": "Represents the ductility of the deposit as measured by the strain at fracture.",
                "Residual Strength": "Represents the strength retained by the deposit after exposure to corrosive environments.",
                "Stress/Strain Behavior": "Describes the overall mechanical response of the sprayed deposit, as evidenced by its stress\u2013strain curve.",
                "Deposit Density": "Represents the consolidated density of the cold-sprayed deposit.",
                "Grain Size": "Represents the average grain size within the sprayed deposit as observed from microstructural analysis.",
                "Microstructural Features": "Describes characteristic aspects such as precipitate distribution and grain morphology within the deposit.",
                "SCC Performance": "Represents the resistance of the sprayed deposit to stress corrosion cracking based on exposure tests.",
                "Fracture Mode": "Indicates the predominant failure mechanism of the deposit, such as ductile rupture or intergranular cracking."
            },
            "standards": {
                "ASTM E8": "Standard Test Method for Tension Testing of Metallic Materials.",
                "ASTM G64-99": "Standard Classification of Resistance to Stress-Corrosion Cracking of Heat-Treatable Aluminum Alloys.",
                "ASTM B117": "Standard Practice for Operating Salt Spray (Fog) Apparatus.",
                "ASTM E112-12": "Standard Test Methods for Determining Average Grain Size."
            }
        },
        "metadata": {
            "Title": "Stress Corrosion Cracking Resistance of Cold-Sprayed Al 6061 Deposits Using a Newly Developed Test Fixture",
            "Abstract": "<jats:p>The stress corrosion cracking (SCC) response of Al 6061 bulk deposits produced by high-pressure cold spray (HPCS) was investigated and compared to commercial wrought Al 6061-T6 material. Representative tensile coupons were stressed to 25%, 65% and 85% of their respective yield strength and exposed to ASTM B117 salt fog for 90 days. After exposure, the samples were mechanically tested to failure, and subsequently investigated for stress corrosion cracking via optical and scanning electron microscopy with energy-dispersive X-ray spectroscopy (EDS). The results were compared to the wrought Al 6061-T6 properties and correlated with the observed microstructures. Wrought samples showed the initiation of stress corrosion cracking, while the cold-sprayed deposits appeared to be unaffected or affected by general corrosion only. Optical microscopy revealed evidence of stress corrosion cracking in the form of intergranular corrosion in the wrought samples, while no significant corrosion was observed in the cold-sprayed deposits. Fractography revealed wrought samples failed due to multiple mechanisms, with predominant cleavage and intergranular failure, but cold-sprayed samples only failed by ductile dimple rupture. The difference in SCC response between the differently processed materials is attributed to the documented benefits of the cold spray process, which includes maintaining fine grain structure of the feedstock powder and high density after consolidation, low oxidation, and work hardening effect.</jats:p>",
            "Date Published": "2019-7-17",
            "DOI": "10.3390/coatings9070445",
            "Number of References": 49,
            "Number of Citations": 3,
            "Journal": "Coatings",
            "Date Updated (of metadata)": "2025-02-21T17:54:36Z"
        }
    },
    {
        "filename": "Article_5.pdf",
        "title": "Mechanical properties and growth mechanisms of long and small fatigue cracks in as-deposited bulk cold spray Al-6061",
        "link": "https://www.sciencedirect.com/science/article/pii/S0142112324000100?casa_token=8BTO0UmGmO4AAAAA:mj-VECrBp7wwXChuFvC8pE1aM0gwtxZXrvXa8j56GwyanemaThTVF6jtUgEd1h33433Bo2_c",
        "extractedText": {
            "pre_sprayed_material_properties": {
                "Powder Material Name": "The generic designation of the metal powder used as feedstock for deposition.",
                "Powder Supplier": "The source or vendor of the powder, including any associated pre\u2010processing such as heat treatment or gas classification.",
                "Powder Handling Condition": "The environmental or handling condition of the powder prior to deposition, such as exposure to humid air."
            },
            "experimental_parameters": {
                "Deposition System": "The equipment configuration used for the cold spray deposition process.",
                "Process Gas Type": "The type of gas used to accelerate the powder particles during the deposition.",
                "Main Gas Temperature Set Point": "The controlled temperature at which the primary process gas is maintained during deposition.",
                "Pressure Set Point": "The specified pressure condition applied during the deposition process.",
                "Carrier Gas Flow Rate": "The actual flow rate of the gas used to carry the powder particles in the system.",
                "Powder Feed RPM": "The rotational speed setting of the powder feeder during the deposition process.",
                "Powder Feeder Vibrator Status": "The operational state of the powder feeder vibrator during deposition.",
                "Robot Traverse Velocity": "The speed at which the spray gun or robotic system moves across the substrate during deposition.",
                "Index Spacing": "The set spatial interval between consecutive deposition passes or points.",
                "Fatigue Test Frequency": "The cyclical frequency used during fatigue crack growth testing.",
                "Stress Ratio": "The ratio of minimum to maximum stress applied during fatigue testing.",
                "Initial Notch Length": "The initial length of the notch created in testing specimens for fatigue crack growth experiments.",
                "Initial Notch Width": "The width of the notch introduced into fatigue testing specimens.",
                "K-Gradient": "The prescribed gradient of the stress intensity factor applied during fatigue crack growth tests.",
                "Load Increment": "The incremental increase in load applied during fatigue testing until crack propagation is detected.",
                "Testing Temperature": "The ambient temperature at which the mechanical tests, including tensile and fatigue tests, are performed.",
                "Relative Humidity": "The range of humidity conditions maintained in the laboratory during testing.",
                "Tensile Specimen Geometry": "The defined physical dimensions and shape of the specimens used for tensile testing.",
                "Compact Tension Specimen Geometry": "The defined physical dimensions and shape of the compact tension specimens used in fatigue testing.",
                "Lap-Shear Specimen Geometry": "The defined physical dimensions and shape of the lap-shear specimens used for interface strength evaluation.",
                "Surface Flaw Tension Specimen Geometry": "The configuration and dimensions of the specimens used to study small fatigue crack growth behavior."
            },
            "sprayed_properties": {
                "Yield Strength": "The stress level at which the sprayed material begins to undergo plastic deformation.",
                "Ultimate Tensile Strength": "The maximum stress the sprayed material can withstand before failure.",
                "Young's Modulus": "The measure of stiffness of the sprayed material as determined from tensile tests.",
                "Elongation": "The strain at fracture, representing the ductility of the sprayed deposit.",
                "Deposited Particle Size": "The average size of the powder particles present within the as-deposited coating.",
                "Grain Size": "The average size of the metallurgical grains found in the sprayed material.",
                "Fatigue Crack Growth Threshold (Applied)": "The initial threshold stress intensity factor at which fatigue crack growth begins in the sprayed material.",
                "Fatigue Crack Growth Threshold (Effective)": "The fatigue crack growth threshold value after correcting for crack closure effects.",
                "Fracture Toughness": "The resistance of the sprayed material to crack propagation under applied stress.",
                "Paris Law Constant C": "The constant in the Paris law equation that characterizes the rate of fatigue crack growth.",
                "Paris Law Exponent m": "The exponent in the Paris law that defines the sensitivity of crack growth rate to changes in stress intensity.",
                "Maximum Stress Intensity Factor (Kmax)": "The peak stress intensity factor reached during fatigue testing of the sprayed deposit.",
                "Strain Energy Release Rate": "The rate at which energy is released per unit area during crack propagation in the sprayed material.",
                "Interface Shear Strength": "The measure of the adhesive strength at the interface between the sprayed deposit and the substrate."
            },
            "standards": {
                "ASTM E8": "The standard test method for tension testing of metallic materials.",
                "ASTM E647": "The standard test method for measuring fatigue crack growth rates.",
                "ASTM D1002": "The standard test method for determining the apparent shear strength of single-lap-joint adhesively bonded metal specimens.",
                "ASTM E112": "The standard test method for determining average grain size in metallic materials.",
                "MIL-J-24445A": "A Department of Defense military specification related to lap-shear testing of metal joints."
            }
        },
        "metadata": {
            "Title": "Mechanical properties and growth mechanisms of long and small fatigue cracks in as-deposited bulk cold spray Al-6061",
            "Abstract": "NA",
            "Date Published": "2024-4",
            "DOI": "10.1016/j.ijfatigue.2024.108152",
            "Number of References": 31,
            "Number of Citations": 4,
            "Journal": "International Journal of Fatigue",
            "Date Updated (of metadata)": "2025-03-19T13:14:30Z"
        }
    },
    {
        "filename": "Article_6.pdf",
        "title": "",
        "link": "",
        "extractedText": "",
        "metadata": "No title provided"
    },
    {
        "filename": "Article_7.pdf",
        "title": "Ultrasonic and conventional fatigue behavior, strain rate sensitivity, and structural design methods for wrought and cold spray Al-6061",
        "link": "https://www.sciencedirect.com/science/article/pii/S2589152924002837?casa_token=5HQzxvgo2O8AAAAA:O8u_M_S5MkcKPLks84R_nScdX6WK4uCwQ_r45fkAdyTg7jwfuWRRUUQoJPq7Pzl23IHpvoXb",
        "extractedText": {
            "pre_sprayed_material_properties": {
              "Powder Material Name": "The generic name identifying the powder used as the feedstock for deposition.",
              "Powder Material Composition": "The generic compositional information of the powder material before processing.",
              "Powder Material Source": "The supplier or origin from which the powder material is acquired.",
              "Powder Pre-treatment Temperature": "The temperature setting used during the heat treatment of the powder prior to deposition.",
              "Powder Pre-treatment Duration": "The time period for which the powder is heat treated before spraying.",
              "Powder Storage Condition": "The environmental condition in which the powder is maintained after pre-treatment.",
              "Powder Agglomerate Breakup Method": "The process applied to separate and de-agglomerate the powder particles prior to spraying.",
              "Powder Particle Size Distribution D10": "The lower percentile value indicating the smallest particle sizes present in the powder distribution.",
              "Powder Particle Size Distribution D50": "The median value representing the typical particle size within the powder distribution.",
              "Powder Particle Size Distribution D90": "The upper percentile value indicating the largest particle sizes present in the powder distribution."
            },
            "experimental_parameters": {
              "Cold Spray System": "The overall system and setup used to perform the cold spray deposition process.",
              "Process Gas": "The type of gas used to propel the powder particles during the deposition process.",
              "Main Gas Temperature Set Point": "The target temperature at which the main process gas is maintained during spraying.",
              "Pressure Set Point": "The designated pressure level maintained during the cold spray deposition.",
              "Carrier Gas Flow Rate": "The volumetric flow rate of the carrier gas used to transport the powder particles.",
              "Powder Feed RPM": "The rotational speed of the powder feed mechanism during the deposition process.",
              "Nozzle Type": "The classification or design of the nozzle employed to accelerate the powder particles.",
              "Nozzle Throat Diameter": "The diameter of the nozzle throat that influences the initial acceleration of particles.",
              "Nozzle Exit Diameter": "The diameter at the nozzle exit which affects the spray dynamics and particle velocity.",
              "Nozzle Total Length": "The overall length of the nozzle that contributes to the particle acceleration process.",
              "Applicator Type": "The design of the applicator used to direct the sprayed material onto the substrate.",
              "Standoff Distance": "The distance maintained between the nozzle and the substrate during deposition.",
              "Spray Angle": "The angle at which the powder particles are directed towards the substrate.",
              "Raster Pattern": "The predetermined movement pattern used to cover the substrate during spraying.",
              "Specimen Geometry": "The geometric configuration of the test specimens (e.g., round or flat) used during the experiment.",
              "Test Frequency": "The cyclic frequency at which fatigue tests are conducted, encompassing conventional and ultrasonic methods.",
              "Strain Rate": "The rate at which deformation is applied during tensile and fatigue testing.",
              "Surface Preparation Method": "The set of procedures used to prepare and polish specimen surfaces prior to testing.",
              "Tensile Testing Method": "The standardized approach and equipment used for measuring tensile properties of the material.",
              "Instrumentation": "The measurement devices and software employed to acquire and analyze experimental data."
            },
            "sprayed_properties": {
              "Deposited Particle Size": "The average size of particles present in the deposited, as-sprayed material.",
              "Deposited Grain Size": "The characteristic grain size within the microstructure of the sprayed deposit.",
              "Youngs Modulus": "The stiffness of the sprayed material as determined by tensile testing.",
              "Yield Strength": "The stress level at which the sprayed material begins to exhibit plastic deformation.",
              "Ultimate Tensile Strength": "The maximum stress the sprayed material can sustain before failure.",
              "Elongation": "The degree of plastic deformation the sprayed material undergoes before fracturing.",
              "Toughness": "The ability of the sprayed material to absorb energy and deform plastically before breaking.",
              "Fatigue Lifetime": "The number of cycles the sprayed material can endure under cyclic loading before failure.",
              "Fatigue Frequency Sensitivity": "The dependence of the sprayed material’s fatigue performance on the testing frequency.",
              "Residual Stress": "The inherent stress state on the surface of the sprayed deposit resulting from the deposition process.",
              "Surface Roughness": "The texture and smoothness of the sprayed deposit's surface as influenced by processing conditions.",
              "Crack Initiation Mode": "The characteristic mechanism or location where cracks first appear in the sprayed material."
            },
            "standards": {
              "ASTM E8": "A standard that specifies the test methods for tensile testing of metallic materials.",
              "ASTM E466": "A standard practice outlining the procedures for conducting constant amplitude axial fatigue tests.",
              "ASTM E112": "A standard test method used for determining the average grain size in metallic materials.",
              "MIL-HDBK-5G": "A military handbook that provides guidelines for metallic materials and elements in aerospace structures."
            }
        },
        "metadata": {
            "Title": "Ultrasonic and Conventional Fatigue Behavior, Strain Rate Sensitivity, and Structural Design Methods for Wrought and Cold Spray Al-6061",
            "Abstract": "NA",
            "Date Published": "2024",
            "DOI": "10.2139/ssrn.4733158",
            "Number of References": 0,
            "Number of Citations": 0,
            "Journal": "NA",
            "Date Updated (of metadata)": "2024-02-21T00:27:40Z"
        }
    },
    {
        "filename": "Article_8.pdf",
        "title": "Effects of processing on microstructure evolution and fatigue crack growth mechanisms in cold-spray 6061 aluminum alloy",
        "link": "https://www.sciencedirect.com/science/article/pii/S0142112318300057?casa_token=3YwKzePCiMIAAAAA:ZsYbulYjY9o2oLWQMV5C0JupLhDY34P63EjyWVH_ARFIdWB9uU6bo8SvIdCtGz9qfVzDROjM",
        "extractedText": { 
            "pre_sprayed_material_properties": { 
                "Powder Material Name": "Generic placeholder for the name of the powder feedstock used before spraying.", 
                "Powder Material Composition": "Represents the elemental or chemical makeup of the powder material used prior to deposition.", 
                "Powder Material Source": "Identifies the supplier or origin of the powder feedstock employed in the process.", 
                "Powder Particle Size": "Indicates the average diameter of the powder particles before spraying.", 
                "Powder Particle Size Distribution": "Describes the variation or spread in sizes among the powder particles used for deposition." 
            }, 
            "experimental_parameters": { 
                "Substrate Material": "Specifies the base material onto which the powder is deposited.", 
                "Substrate Grit Blasting": "Describes the process parameters for preparing the substrate surface using grit blasting.", 
                "Substrate Cleaning Method": "Specifies the cleaning procedure applied to the substrate prior to deposition.", 
                "Nozzle Throat Diameter": "Represents the diameter of the nozzle throat used in the deposition process.", 
                "Nozzle Weight": "Indicates the weight of the nozzle, which can influence the spray dynamics.", 
                "Gas Type": "Specifies the type of gas utilized to accelerate the powder particles in the cold-spray process.", 
                "Gas Pressure and Flow Rate": "Represents the operational pressure and volumetric flow conditions of the gas during deposition.", 
                "Heater Set Points": "Defines the temperature settings for both the gun heater and the pre-heater during spraying.", 
                "Powder Feed Rate": "Specifies the rate at which powder is delivered into the spray system.", 
                "Spray Distance": "Indicates the gap between the spray gun and the substrate during deposition.", 
                "Spray Gun Velocity": "Represents the speed at which the spray gun operates during the deposition process.", 
                "Specimen Dimensions": "Describes the geometric parameters of test specimens prepared for mechanical evaluation.", 
                "Test Frequency": "Specifies the cyclic loading frequency applied during fatigue testing." 
            }, 
            "sprayed_properties": { 
                "Microhardness": "Represents the hardness of the sprayed deposit as determined by micro-indentation tests.", 
                "Yield Strength": "Indicates the stress level at which the sprayed material begins to deform plastically.", 
                "Ultimate Tensile Strength": "Represents the maximum stress that the sprayed material can sustain before failure.", 
                "Elongation at Failure": "Describes the ductility of the sprayed deposit by measuring the strain at fracture.", 
                "Elastic Modulus": "Indicates the stiffness of the sprayed material as measured in tensile tests.", 
                "Residual Stress": "Represents the internal stresses retained within the sprayed material following deposition.", 
                "Fatigue Crack Growth Threshold": "Specifies the minimum stress intensity factor range necessary to initiate crack propagation in the sprayed material.", 
                "Maximum Applied Stress Intensity Factor": "Represents the peak stress intensity experienced by the sprayed deposit during testing.", 
                "Paris Law Constant (C and m)": "Describes the empirical constants used in the Paris law to predict fatigue crack growth rates in the sprayed material.", 
                "Fatigue Crack Growth Rate": "Indicates the rate at which a crack propagates in the sprayed deposit under cyclic loading conditions." 
            }, 
            "standards": { 
                "ASTM E3-11": "Standard that outlines the procedures for preparing metallographic specimens.", 
                "ASTM E112-12": "Standard test method for determining the average grain size of metallic materials.", 
                "ASTM E384-11": "Standard test method for measuring hardness using micro-indentation techniques.", 
                "ASTM E8/E8M-16a": "Standard test methods for tensile testing of metallic materials.", 
                "ASTM E647-15": "Standard test method for measuring fatigue crack growth rates in materials." 
            } 
        },
        "metadata": {
            "Title": "Effects of processing on microstructure evolution and fatigue crack growth mechanisms in cold-spray 6061 aluminum alloy",
            "Abstract": "NA",
            "Date Published": "2018-5",
            "DOI": "10.1016/j.ijfatigue.2018.01.006",
            "Number of References": 46,
            "Number of Citations": 57,
            "Journal": "International Journal of Fatigue",
            "Date Updated (of metadata)": "2025-03-19T14:00:55Z"
        }
    },
    {
        "filename": "Article_9.pdf",
        "title": "Quantitative Nondestructive Evaluation of Cold Spray Manufactured Aluminum Alloy 6061 and Copper Samples",
        "link": "https://link.springer.com/article/10.1007/s11666-024-01738-y",
        "extractedText": { 
            "pre_sprayed_material_properties": { 
                "Material Name": "A generic placeholder representing the name of the feedstock material used in the process.", 
                "Material Composition": "A generic placeholder indicating the chemical makeup or alloy composition of the feedstock material.", 
                "Powder Material Source": "Specifies the supplier or origin from which the powder was procured.", 
                "Powder Particle Size Distribution": "Represents the range of particle sizes in the powder feedstock, often reported via percentiles (D10, D50, D90).", 
                "Powder Particle Shape (Sphericity)": "Indicates the roundness and uniformity of the powder particles, affecting flow and deposition behavior.", 
                "Powder Density": "Defines the non-porous density of the powder material used prior to deposition.", 
                "Powder Classification": "Categorizes the powder based on predefined criteria such as particle size, shape, and treatment." 
            }, 
            "experimental_parameters": { 
                "Process Gas Type": "Indicates the type of gas used as the carrier in the cold spray process.", 
                "Process Gas Pressure": "Specifies the pressure of the gas applied during the spraying operation.", 
                "Process Gas Temperature": "Specifies the operating temperature of the process gas during spraying.", 
                "Particle Impact Velocity": "The velocity at which powder particles strike the substrate during deposition.", 
                "Impact Velocity Ratio (V/V_cr)": "The ratio of the actual particle impact velocity to the critical velocity required for bonding.", 
                "Nozzle Throat Diameter": "Defines the diameter of the nozzle throat, which influences gas flow and particle acceleration.", 
                "Nozzle Expansion Section Length": "Specifies the length of the nozzle's expansion section that affects spray dynamics.", 
                "Substrate Preparation Method": "Describes the surface treatment or cleaning procedures applied to the substrate before deposition.", 
                "Deposition Area Dimensions": "Specifies the physical dimensions of the area onto which material is deposited.", 
                "Raster Pattern Line Separation": "Indicates the spacing between lines in the deposition raster pattern used to build up the coating.", 
                "Ultrasound Probe Frequency": "The operating frequency of the ultrasound probe used during nondestructive testing of the deposit.", 
                "Eddy Current Testing Frequency": "The frequency at which the eddy current system operates to measure electrical conductivity." 
            }, 
            "sprayed_properties": { 
                "Deposition Thickness": "The measured thickness of the coating or deposit formed on the substrate.", 
                "Porosity": "Represents the percentage of voids or defects within the sprayed deposit that affect its overall density.", 
                "Ultrasound Wave Velocity": "The speed at which ultrasound waves travel through the deposit, serving as an indicator of material quality and bonding.", 
                "Electrical Conductivity": "Measures the ability of the sprayed deposit to conduct electricity, often evaluated via eddy current methods.", 
                "Hardness": "Indicates the resistance of the deposit to deformation, typically measured using micro-Vickers or similar methods.", 
                "Tensile Strength": "The maximum stress that the sprayed deposit can withstand before failure under tensile loading.", 
                "Elastic Modulus": "A measure of the stiffness of the sprayed deposit as determined by ultrasound or tensile testing methods.", 
                "Ductility": "Describes the ability of the sprayed deposit to undergo plastic deformation prior to fracture.", 
                "Interfacial Bonding Quality": "Assesses the adhesion and mechanical interlocking between deposited particles within the coating.", 
                "Fractography Features": "Describes the characteristics of the fracture surfaces, providing insights into failure modes and microstructural quality." 
            }, 
            "standards": { 
                "Standard MIL-DTL-32495": "A standard that specifies requirements for powders used in cold spray deposition processes.", 
                "Standard ASTM-F3049-14": "A guide for characterizing properties of metal powders used in additive manufacturing.", 
                "Standard ASTM-F3049-14R21": "A revised standard guide for the characterization of metal powders utilized in additive manufacturing.", 
                "Standard ASTM F42": "A standard addressing design and qualification procedures for additive manufacturing processes.", 
                "Standard ISO TC261": "An ISO standard from the technical committee dedicated to additive manufacturing.", 
                "Standard ASTM E3": "A standard guide for the preparation of metallographic specimens for material analysis.", 
                "Standard ASTM E8/E8M-16a": "A standard test method for tension testing of metallic materials to evaluate mechanical properties." 
            } 
        },
        "metadata": {
            "Title": "Quantitative Nondestructive Evaluation of Cold Spray Manufactured Aluminum Alloy 6061 and Copper Samples",
            "Abstract": "<jats:title>Abstract</jats:title><jats:p>Cold spray (CS) is a solid-state process for depositing thick layers of material via the successive high-velocity impact of powder particles onto a solid surface, which leads to high rates of deformation, interparticle bonding, and coating build-up. Although CS is finding commercial utilization in non-load-bearing repair and coating applications, clear nondestructive characterization procedures are necessary to realize its potential in load-bearing structural applications. In this study, the viability of electrical conductivity and through thickness ultrasound wave velocity measurement methods was studied to serve as a means for nondestructive quantitative measurement for quality control in CS and potentially other additive manufacturing (AM) methods. Eddy current, ultrasound, porosity, hardness, and uniaxial tensile strength tests were conducted on CS deposited layers of aluminum alloy 6061 and copper on aluminum alloy 6061 and commercially pure copper substrates, respectively. CS gas dynamic parameters were intentionally and systematically varied to result in corresponding discrete differences in mechanical properties of deposits. Ultrasound measurements of longitudinal wave velocity and eddy current electrical conductivity measurements showed good correlation with process conditions, microstructural characterization results, and destructive mechanical tests (hardness, tensile). The results of this work show that ultrasound wave velocity and electrical conductivity correlate well with increased particle impact velocity in CS deposited aluminum and copper blocks, which evidently show an incremental decrease in porosity, increase in hardness, and increase in tensile strength. The outlined ultrasound and eddy current nondestructive testing methods present effective means for quantitative assessment of cold spray deposited structures while intact with the substrate.</jats:p>",
            "Date Published": "2024-2-28",
            "DOI": "10.1007/s11666-024-01738-y",
            "Number of References": 53,
            "Number of Citations": 1,
            "Journal": "Journal of Thermal Spray Technology",
            "Date Updated (of metadata)": "2025-02-21T11:45:07Z"
        }
    },
    {
        "filename": "Article_10.pdf",
        "title": "Surface states of gas-atomized Al 6061 powders \u2013 Effects of heat treatment",
        "link": "sciencedirect.com/science/article/pii/S0169433220324004?via%3Dihub",
        "extractedText": { 
            "pre_sprayed_material_properties": { 
                "Material Name": "Represents the generic identifier of the powder feedstock material.", 
                "Material Composition": "Represents the generic alloy composition of the powder feedstock.", 
                "Powder Particle Size": "Represents the average particle diameter and size distribution of the powder.", 
                "Surface Oxide Layer Thickness": "Indicates the thickness of the oxide layer formed on the powder surface.", 
                "Surface Oxide Layer Structure": "Describes the structural features of the oxide layer, including its distinct sub-layers.", 
                "Surface Oxide Sub-layer Composition": "Indicates the compositional differentiation between the outer Mg-rich sub-layer and the inner Mg-lean sub-layer of the oxide layer.", 
                "Powder Production Method": "Describes the process used to produce the powder feedstock, such as gas atomization.", 
                "Powder Surface Morphology": "Describes the topographical features of the powder surface, including ripples, grooves, and satellite particles." 
            }, 
            "experimental_parameters": { 
                "Heat Treatment Temperature": "Represents the temperature at which the powder is heat treated.", 
                "Heat Treatment Duration": "Represents the length of time the powder is held at the designated heat treatment temperature.", 
                "Atmosphere During Heat Treatment": "Specifies the gaseous environment, such as air or an inert gas, used during the heat treatment process.", 
                "Homogenization Temperature Profile": "Describes the multi-stage temperature profile applied during the homogenization heat treatment process.", 
                "XPS Measurement Parameters": "Includes the instrumental settings such as the radiation source, binding energy calibration, and pass energies used in XPS analysis.", 
                "Electron Microscopy Parameters": "Specifies the accelerating voltage and related settings used during SEM/TEM imaging.", 
                "FIB Milling Parameters": "Describes the procedures and settings used during focused ion beam milling for sample preparation." 
            }, 
            "sprayed_properties": { 
                "Cold Spray Deposit Microstructure": "Describes the microstructural features observed in the cold-sprayed deposits.", 
                "Splat Mechanical Behavior": "Represents the mechanical response and deformation characteristics of individual cold-sprayed splats.", 
                "Bonding Characteristics": "Describes the quality of metallurgical bonding between particles in the cold spray deposits." 
            }, 
            "standards": { 
                "Material Standard (T6 Condition)": "Represents a standard reference material in a generic T6 condition used for calibration.", 
                "Material Standard (Rocksalt Structure)": "Represents a standard reference material with a generic rocksalt structure used for calibration.", 
                "Material Standard (Spinel Structure)": "Represents a standard reference material with a generic spinel structure used for calibration.", 
                "Material Standard (Sapphire Structure)": "Represents a standard reference material with a generic sapphire structure used for calibration.", 
                "Material Standard (Powder Form)": "Represents a standard reference material provided in powder form used for calibration." 
            } 
        },
        "metadata": {
            "Title": "Surface states of gas-atomized Al 6061 powders \u2013 Effects of heat treatment",
            "Abstract": "NA",
            "Date Published": "2020-12",
            "DOI": "10.1016/j.apsusc.2020.147643",
            "Number of References": 55,
            "Number of Citations": 19,
            "Journal": "Applied Surface Science",
            "Date Updated (of metadata)": "2025-02-21T04:02:20Z"
        }
    }
}



SyntaxError: closing parenthesis '}' does not match opening parenthesis '[' on line 3 (3339436913.py, line 530)